PLEASE NOTE THAT IF YOU WANT TO SEE MY FINAL PREDICTIONS TABLE, YOU NEED TO UNCOMMENT THE PRINT(STORE) IN THE FUNCTION
N.B: All questions are explained in more details in the 1 page pdf!

In [1]:
#importing the required libraries
import numpy as np
import pandas as pd
import math
from sklearn.datasets import load_iris
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
#reading the data from the excel file
df=pd.read_excel('exerciseCB.xlsx',sheet_name='CB - Simply Unary',header=1,index_col=0)

In [3]:
#creating 3 different dataframes from the 3 different tables in the excel file
q_data = df.iloc[:20,:10]
user_data=df.iloc[:20,13:17].fillna(0)
answer_data=df.iloc[:20,18:22].fillna(0)

In [10]:
#Simply Unary
# Dot product of the user feedback with each topic contained in the questions to which the user gave feedback
# Each user ends up with a numeric value for each topic --> user profile
# Then I multiply the user profile with the topics contained in each question using the cosine similarity 
# I then returned the top 5 questions predicted for each user
def SimplyUnary(qdata, userdata):
    #initializing and defining dataframes for later use
    store=pd.DataFrame()
    check=qdata.copy()
    check_2=qdata.copy()
    j=1
    #iterating over every topic in the questions for every User that we have using 2 for loops
    for i in userdata.columns:
        for col in qdata.columns:
            #for the non value columns, I multiplied each topic column with the user feedback and summed the result
            check_2[col]=np.where(qdata.loc[:,col]==0,qdata.loc[:,col],userdata.loc[:,i]*qdata.loc[:,col])
            user_profile=pd.DataFrame(check_2.sum(axis=0)).transpose()
            #then I multiplied the obtained sum (as a vector) with the topic each question has, for every user
            #using the cosine similarity
            mult=cosine_similarity(check,user_profile).sum(axis=1)
        store[j]=mult
        #clearing the values used for the previous user in order to use them for the next one
        user_profile=[]
        mult=[]
        j=j+1
    store.index=userdata.index
    # sorting the values contained for every user, and generating the 'top5' questions for every user
    top = pd.DataFrame()
    for p in store.columns:
        #if the user is new we cannot generate recommendations using this method and hence we return this msg
        if ((store.loc[:,p].mean())==0 and (store.loc[:,p].std())==0):
            top[p]='No Data Available'
        else:
            rank = store.loc[:,p].sort_values(ascending = False)
            top[p]=rank.index
    top5=top.loc[:4,:]
    top5.columns=userdata.columns
    store.columns=userdata.columns
    print(store)
    return top5

In [11]:
SimplyUnary(q_data, user_data)

                  User 1        User 2        User 3  User 4
question1   3.903600e-01 -2.981424e-01 -2.927700e-01     0.0
question2  -4.364358e-01  8.333333e-01  0.000000e+00     0.0
question3   2.519763e-01  1.993498e-17 -3.779645e-01     0.0
question4  -3.273268e-01  6.666667e-01 -2.182179e-01     0.0
question5  -1.259882e-01  9.622504e-02  2.519763e-01     0.0
question6   4.629100e-01  1.178511e-01 -3.086067e-01     0.0
question7  -1.543033e-01  2.357023e-01 -1.543033e-01     0.0
question8  -2.182179e-01  3.333333e-01  1.091089e-01     0.0
question9   4.629100e-01 -2.357023e-01 -4.629100e-01     0.0
question10 -3.779645e-01  9.622504e-02 -1.333738e-17     0.0
question11  1.333738e-17  9.622504e-02  1.259882e-01     0.0
question12  5.039526e-01 -3.849002e-01 -7.559289e-01     0.0
question13 -2.182179e-01  5.833333e-01 -1.091089e-01     0.0
question14 -2.182179e-01  5.833333e-01  2.182179e-01     0.0
question15  0.000000e+00  3.333333e-01 -6.546537e-01     0.0
question16  7.559289e-01

,User 1,User 2,User 3,User 4
0,question16,question17,question5,No Data Available
1,question12,question2,question14,No Data Available
2,question6,question4,question19,No Data Available
3,question9,question14,question11,No Data Available
4,question1,question13,question8,No Data Available


In [ ]:
#Unit Weight
#in this function, we will repeat the Simply Unary calculations BUT ALSO assigned a weight to each question 
#this weight is calculated by taking into account the number of topics a question has
#by dividing the topic appearance by the total number of topics a question has
# I then returned the top 5 questions predicted for each user
def UnitWeight(qdata, userdata):  
    # calculating the weight of each question, by dividing with the sum of the topic occurence for every question
    test_2=[]
    for i in qdata.index:
        test_1=1/sum(qdata.loc[i,:])
        test_2.append(test_1)
    #initializing and defining dataframes for later use
    check=qdata.copy()
    check_2=qdata.copy()
    j=1
    store=pd.DataFrame()
    #iterating over every topic in the questions for every User that we have using 2 for loops
    for i in userdata.columns:
        for col in qdata.columns:
            #for the non value columns, I multiplied each topic column with the user feedback and I also multiplied
            #by the weight that I previously calculated for each question and summed the result
            check_2[col]=np.where(qdata.loc[:,col]==0,qdata.loc[:,col],userdata.loc[:,i]*qdata.loc[:,col]*test_2)
            user_profile=pd.DataFrame(check_2.sum(axis=0)).transpose()
            #then I multiplied the obtained sum (as a vector) with the topic each question has, for every user
            #using the cosine similarity
            mult=cosine_similarity(check, user_profile).sum(axis=1)
        store[j]=mult
        #clearing the values used for the previous user in order to use them for the next one
        user_profile=[]
        standardized_X=[]
        mult=[]
        j=j+1
    # sorting the values contained for every user, and generating the 'top5' questions for every user
    store.index=userdata.index    
    top = pd.DataFrame()
    for p in store.columns:
        #if the user is new we cannot generate recommendations using this method and hence we return this msg
        if ((store.loc[:,p].mean())==0 and (store.loc[:,p].std())==0):
            top[p]='No Data Available'
        else:
            rank = store.loc[:,p].sort_values(ascending = False)
            top[p]=rank.index
    top5=top.loc[:4,:]
    top5.columns=userdata.columns
    store.columns=userdata.columns
    #print(store)
    return top5

In [ ]:
UnitWeight(q_data, user_data)

In [ ]:
#IDF
# In addtition to the steps taken in the Unit Weight, we now take the relevance of each topic into account
#the higher the number of questions asked for each topic, the lower its relevance is
# I then returned the top 5 questions predicted for each user
def IDF(qdata, userdata):
    #initializing and defining dataframes for later use
    test_3=[]
    test_2=[]
    check=qdata.copy()
    # calculating the weight of each question, by dividing with the sum of the topic occurence for every question
    for i in qdata.index:
        test_1=1/sum(qdata.loc[i,:])
        test_2.append(test_1)
    #summing the occurence of each topic and then computing the LOG for 20 over that sum in order to have IDF
    for i in qdata.columns:
        test_1=np.log10(20/sum(qdata.loc[:,i]))
        test_3.append(test_1)
    test_3=pd.DataFrame(test_3).transpose()
    test_3.columns=qdata.columns
    check_2=qdata.copy()
    j=1
    store=pd.DataFrame()
    #iterating over every topic in the questions for every User that we have using 2 for loops
    for i in userdata.columns:
        for col in qdata.columns:
            #for the non value columns, I multiplied each topic column with the user feedback and I also multiplied
            #by the weight that I previously calculated for each question and by IDF before summing the result
            check_2[col]=np.where(qdata.loc[:,col]==0,qdata.loc[:,col],userdata.loc[:,i]*qdata.loc[:,col]*test_2*test_3.loc[0,col])
            user_profile=pd.DataFrame(check_2.sum(axis=0)).transpose()
            #then I multiplied the obtained sum (as a vector) with the topic each question has, for every user
            #using the cosine similarity
            mult=cosine_similarity(check, user_profile).sum(axis=1)
        store[j]=mult
        user_profile=[]
        mult=[]
        j=j+1
    store.index=userdata.index 
    # sorting the values contained for every user, and generating the 'top5' questions for every user
    top = pd.DataFrame()
    for p in store.columns:
        rank = store.loc[:,p].sort_values(ascending = False)
        top[p]=rank.index
    top5=top.loc[:4,:]
    top5.columns=userdata.columns
    store.columns=userdata.columns
    #print(store)
    return top5

In [ ]:
IDF(q_data, user_data)

In [ ]:
#SwitchedHybrid
# I addition to the steps taken in IDF, we took into account the case of new users who did not give feedback on 
# any question. We computed their recommendations by taking the average of the other user predictions
# I then returned the top 5 questions predicted for each user
def SwitchedHybrid(qdata, userdata):  
    #initializing and defining lists and dataframes for later use
    test_3=[]
    test_2=[]
    check=qdata.copy()
    # calculating the weight of each question, by dividing with the sum of the topic occurence for every question
    for i in qdata.index:
        test_1=1/sum(qdata.loc[i,:])
        test_2.append(test_1)
    #summing the occurence of each topic and then computing the LOG for 20 over that sum in order to have IDF
    for i in qdata.columns:
        test_1=np.log10(20/sum(qdata.loc[:,i]))
        test_3.append(test_1)
    test_3=pd.DataFrame(test_3).transpose()
    test_3.columns=qdata.columns
    test_3
    j=1
    store=pd.DataFrame()
    check_2=qdata.copy()
    #iterating over every topic in the questions for every User that we have using 2 for loops
    for i in userdata.columns:
        #if the user is a new one, we will return the mean of the predictions of other users!
        if ((userdata.loc[:,i].mean())==0 and (userdata.loc[:,i].std())==0):
            store.loc[:,j]=store.loc[:, store.columns != j].mean(axis=1)
        else:
            for col in qdata.columns:
                #for the non value columns, I multiplied each topic column with the user feedback and I also multiplied
                #by the weight that I previously calculated for each question and by IDF before summing the result
                check_2[col]=np.where(qdata.loc[:,col]==0,qdata.loc[:,col],userdata.loc[:,i]*qdata.loc[:,col]*test_2*test_3.loc[0,col])
                user_profile=pd.DataFrame(check_2.sum(axis=0)).transpose()
                #then I multiplied the obtained sum (as a vector) with the topic each question has, for every user
                #using the cosine similarity
                mult=cosine_similarity(check, user_profile).sum(axis=1)
            store[j]=mult
            user_profile=[]
            mult=[]
            j=j+1
    store.index=userdata.index  
    # sorting the values contained for every user, and generating the 'top5' questions for every user
    top = pd.DataFrame()
    for p in store.columns:
        rank = store.loc[:,p].sort_values(ascending = False)
        top[p]=rank.index
    top5=top.loc[:4,:]
    top5.columns=userdata.columns
    store.columns=userdata.columns
    #print(store)
    return top5

In [ ]:
SwitchedHybrid(q_data, user_data)

In [ ]:
user_data.head()

In [ ]:
#Hybrid Challenge
# I decided to create a recommender that takes into account the IDF methods, the new users and also the VOTES GIVEN
# to the answers of the users! Because all previous 4 methods ignored it!
# FOR MORE DETAILS PLEASE CHECK THE 1 PAGE SUBMISSION!
# I then returned the top 5 questions predicted for each user
def HybridChallenge(qdata, userdata, answerdata): 
    #initializing and defining lists and dataframes for later use
    check=qdata.copy()
    userdata=userdata.copy()
    answerdata.columns=userdata.columns
    #we scaled the answers_data in order to have a range of values that match the range of the user feedback
    X_2 = preprocessing.scale(answerdata)
    X_3 = answerdata.copy()
    #since while scaling, all the zeros are transformed to non zero values
    # I created a new table, in order to keep the zeros and replace all non zero values with 1
    for na in X_3.columns:
        for pe in X_3.index:
            if (X_3.loc[pe,na]!=0):
                X_3.loc[pe,na]=1
    #then I multiplied the scaled data(X_2) with the 0 and 1 table in order to get the zeros back 
    X_2=pd.DataFrame(X_2)
    X_4=np.multiply(X_2,X_3)
    X_4.columns=answerdata.columns
    X_4.index=answerdata.index
    answerdata=X_4.copy()
    # I took into account the questions that a user didn't give feedback to, but gave a good or bad answer instead
    # This is based on the fact that if a user gave a good answer to a question, then we as a PLATFORM want to
    # recommend him more of that question, and if he gave a bad feedback then we don't want these questions
    for n in userdata.columns:
        for p in userdata.index:
            if (userdata.loc[p,n]==1):
                if (answerdata.loc[p,n]!=0):
                    userdata.loc[p,n]=userdata.loc[p,n]*answerdata.loc[p,n]
            elif(userdata.loc[p,n]== -1):
                if (answerdata.loc[p,n]<0):
                    userdata.loc[p,n]=(-1)*userdata.loc[p,n]*answerdata.loc[p,n]
                if(answerdata.loc[p,n]>0):
                    userdata.loc[p,n]=(-1)*userdata.loc[p,n]*answerdata.loc[p,n]
            elif(userdata.loc[p,n]== 0):
                if (answerdata.loc[p,n]!=0):
                    userdata.loc[p,n]=answerdata.loc[p,n]  
    #initializing and defining lists and dataframes for later use
    test_3=[]
    test_2=[]
    # calculating the weight of each question, by dividing with the sum of the topic occurence for every question
    for i in qdata.index:
        test_1=1/sum(qdata.loc[i,:])
        test_2.append(test_1)
    #summing the occurence of each topic and then computing the LOG for 20 over that sum in order to have IDF
    for i in qdata.columns:
        test_1=np.log10(20/sum(qdata.loc[:,i]))
        test_3.append(test_1)
    test_3=pd.DataFrame(test_3).transpose()
    test_3.columns=qdata.columns
    test_3
    j=1
    store=pd.DataFrame()
    #iterating over every topic in the questions for every User that we have using 2 for loops
    for i in userdata.columns:
        #if the user is a new one, we will return the mean of the predictions of other users!
        if ((userdata.loc[:,i].mean())==0 and (userdata.loc[:,i].std())==0):
            store.loc[:,j]=store.loc[:, store.columns != j].mean(axis=1)
        else:
            check_2=qdata.copy()
            for col in qdata.columns:
                #for the non value columns, I multiplied each topic column with the user feedback and I also multiplied
                #by the weight that I previously calculated for each question and by IDF before summing the result
                check_2[col]=np.where(qdata.loc[:,col]==0,qdata.loc[:,col],userdata.loc[:,i]*qdata.loc[:,col]*test_2*test_3.loc[0,col])
                user_profile=pd.DataFrame(check_2.sum(axis=0)).transpose()
                #then I multiplied the obtained sum (as a vector) with the topic each question has, for every user
                #using the cosine similarity
                mult=cosine_similarity(check, user_profile).sum(axis=1)
            store[j]=mult
            user_profile=[]
            mult=[]
            j=j+1
    store.index=userdata.index 
    # sorting the values contained for every user, and generating the 'top5' questions for every user
    top = pd.DataFrame()
    for p in store.columns:
        rank = store.loc[:,p].sort_values(ascending = False)
        top[p]=rank.index
    top5=top.loc[:4,:]
    top5.columns=userdata.columns
    store.columns=userdata.columns
    #print(store)
    return top5

In [ ]:
HybridChallenge(q_data, user_data, answer_data)